##### [pygame 프로젝트]

1. 3월 개발 리스트 
   - 기본 설정 
       - pygame 설치 - 17점수까지
       - pip install pygame
       
   - 장애물 개수 추가 
   - 장애물 별 attack 수 변경
   
   
2. 4월 개발 리스트
   - 코드 리펙토링(진행중 - 완료)
   - 시작 화면 추가하기
   
   - 아이템 추가 예정 
   - 보스 추가 예정 
   


In [1]:
"""
====================
화면 변수 설정
====================
"""
import pygame 
import random

# 게임 초기화
pygame.init()

# 화면 너비 및 게임 시간 체크
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 900

# 이미지 위치 
location_url = 'shoot_rock/'

# 화면 색상 변수
screenColor_Black = (0, 0, 0)
screenColor_Red = (255, 0, 0)
screenColor_Green = (0, 255, 0)
screenColor_Blue = (0, 0, 255)
screenColor_White = (255, 255, 255)
screenColor_Gray = (128, 128, 128)
screenColor_Yellow = (255, 255, 0)

# 폰트 변수 생성 
large_font = pygame.font.SysFont('malgungothic', 72)
small_font = pygame.font.SysFont('malgungothic', 36)
score_font = pygame.font.SysFont('Maplestory Bold.ttf', 30)


# 이미지 변수 생성 
background_image = pygame.image.load(location_url + 'planet.png')
fighter_image = pygame.image.load(location_url + 'ship.png')
attack_image = pygame.image.load(location_url + 'missile1.png')
explosion_image = pygame.image.load(location_url + 'explosion.png')

# 장애물
wall_level_1_image = pygame.image.load(location_url + 'wall_level_1.png')
wall_level_2_image = pygame.image.load(location_url + 'wall_level_2.png')

# 음악 생성
pygame.mixer.init()
pygame.mixer.music.load(location_url + 'JLD_-_Lethal_Xcess_Level_4.mp3')
pygame.mixer.music.set_volume(0.5)
pygame.mixer.music.play(-1) # 무한 반복

attack_sound = pygame.mixer.Sound(location_url + 'missile.wav')
attack_sound.set_volume(0.3)

explosion_sound = pygame.mixer.Sound(location_url + 'explosion.wav')
explosion_sound.set_volume(0.3)

game_over_sound = pygame.mixer.Sound(location_url + 'game_over.wav')
game_over_sound.set_volume(0.3)


# 화면 스크린 생성
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) 
clock = pygame.time.Clock() 



pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
====================
공통 DEFAULT 변수 설정
====================
"""
# 기본 ZERO 값 
DEFAULT_ZERO = 0
DEFAULT_TWO = 2

# 화면 시메트릭
DEFAULT_CENTERX = SCREEN_WIDTH // 2
DEFAULT_CENTERY = SCREEN_HEIGHT // 2

# 비행기 이동 기본 값
DEFAULT_FIGHTER_KEY = 5

# ATTACK 기본 값 
DEFAULT_ATTACK = 1
DEFAULT_TWO_ATTACK = 2
DEFAULT_ATTACK_SPPED = 25

# MISS 기본 값 
DEFAULT_MISS = 1

# MISS 종료 기본 값 
DEFAULT_END_MISS = 4

# 랜덤 기본 값
DEFAULT_RANDOM_NUMBER = random.randint(3, 9)

# TOP DOWN 기본 값
DEFAULT_TOP_DOWN_NUMBER = -100

# 벽돌 이미지 랜덤 Y 위치 값
DEFAULT_LEVEL1_SCREEN_WIDTH = SCREEN_WIDTH - wall_level_1_image.get_width()
DEFAULT_LEVEL2_SCREEN_WIDTH = SCREEN_WIDTH - wall_level_2_image.get_width()

# 점수 기본 값 
DEFAULT_SCORE = 1

# 장애물 생성 개수 기본 값
DEFAULT_ROCKS = 3
DEFAULT_DOUBLE_ROCKS = 1

"""
====================
공통 변수 설정
====================
"""
# TOTAL 점수 
TOTAL_SCORE = 0

# DOUBLE ATTACK 기본 값
DOUBLE_ATTACK_COUNT = 0
DOUBLE_ATTACK_CHECK = False

# GAME OVER 기본 값 
GAME_OVER = False
GAME_END = False

# PYGAME 기본 값
PYGAME_END = 0



In [3]:
"""
====================
초기 변수 (변수)
====================
"""
# 변수 및 배열 선언
fighter = fighter_image.get_rect(centerx=SCREEN_WIDTH // 2, bottom=SCREEN_HEIGHT)
rocks = []
double_rocks = []
attacks = []
missiles = []
score = 0
missed = 0 
gameOver = False

#
attacks_tmp = []


# 기본 장애물 ATTACK TMP 변수 
one_attack_tmp_score = 0
one_attack_tmp_rocks = []
one_attack_tmp_attacks = []


# 더블 장애물 ATTACK TMP 변수 
double_attack_tmp_chk = False
double_attack_tmp_score = 0
double_attack_tmp_count = 0
double_attack_tmp_rocks = []
double_attack_tmp_attacks = []




In [4]:
"""
====================
장애물 생성 (변수)
====================
"""
# 바위 생성
for i in range(DEFAULT_ROCKS):
    rock = wall_level_1_image.get_rect(left=random.randint(0, DEFAULT_LEVEL1_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
    dy = DEFAULT_RANDOM_NUMBER
    rocks.append((rock, dy))


# 더블 바위 생성 
for i in range(DEFAULT_DOUBLE_ROCKS):
    double_rock = wall_level_2_image.get_rect(left=random.randint(0, DEFAULT_LEVEL2_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
    dy = DEFAULT_RANDOM_NUMBER
    double_rocks.append((double_rock, dy))
    
    

In [5]:
"""
====================
함수
====================
"""
# 기본 게임 핸들링 처리 함수 
def handlingBranchDefaultGame(event, attacks, PYGAME_END):
    if event.type == pygame.QUIT:
        PYGAME_END = 1
        return attacks, PYGAME_END
    
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_SPACE:
            attack = pygame.Rect(attack_image.get_rect(centerx=fighter.centerx, top=fighter.top))
            attacks.append(attack)
            attack_sound.play()
        
    return attacks, PYGAME_END

# 비행기 핸들링 처리 함수 
def handlingBranchDefaultFighter(fighter, gameOver):
    pressed = pygame.key.get_pressed()
    if pressed[pygame.K_LEFT] and not gameOver:
        fighter.left -= DEFAULT_FIGHTER_KEY
    elif pressed[pygame.K_RIGHT] and not gameOver:
        fighter.left += DEFAULT_FIGHTER_KEY

    return fighter
 

# 기본 장애물 MISS 분기 처리 함수 
def handlingBranchMissRock(rocks, missed):
    for rock, dy in rocks:
        rock.top += dy
        if rock.top > SCREEN_HEIGHT:
            rocks.remove((rock, dy))
            rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top=DEFAULT_TOP_DOWN_NUMBER)
            dy = DEFAULT_RANDOM_NUMBER
            rocks.append((rock, dy))
            missed += DEFAULT_MISS
    return missed


# 더블 장애물 MISS 분기 처리 함수 
def handlingBranchMissDoubleRock(double_rocks, missed):
    for double_rock, dy in double_rocks:
        double_rock.top += dy
        if double_rock.top > SCREEN_HEIGHT:
            double_rocks.remove((double_rock, dy))
            double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top = DEFAULT_TOP_DOWN_NUMBER)
            dy = DEFAULT_RANDOM_NUMBER
            double_rocks.append((double_rock, dy))
            missed += DEFAULT_MISS            
    return missed

# 기본 장애물 Attack 분기 처리 함수 
def handlingBranchAttackRock(rocks, attacks, score):
    for rock, dy in rocks:
        for attack in attacks:
            if attack.colliderect(rock):
                rocks.remove((rock, dy))
                attacks.remove(attack)
                screen.blit(explosion_image, (rock.left, rock.top))
                rock = wall_level_1_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL1_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
                dy = DEFAULT_RANDOM_NUMBER
                rocks.append((rock, dy))
                score += DEFAULT_SCORE
                explosion_sound.play()
    return score


# 더블 장애물 Attack 분기 처리 함수 
def handlingBranchAttackDoubleRock(double_rocks, attacks, score, DOUBLE_ATTACK_COUNT, DOUBLE_ATTACK_CHECK):
    
    for double_rock, dy in double_rocks:
        for attack in attacks:
            if attack.colliderect(double_rock):
                attacks.remove(attack)
                DOUBLE_ATTACK_COUNT += DEFAULT_ATTACK
                print("DOUBLE_ATTACK_COUNT : ", DOUBLE_ATTACK_COUNT)
                
                # 2번 공격했을 경우 
                if DOUBLE_ATTACK_COUNT >= DEFAULT_TWO_ATTACK:    
                    double_rocks.remove((double_rock, dy))
#                   attacks.remove(attack)
                    screen.blit(explosion_image, (double_rock.left, double_rock.top))
                    double_rock = wall_level_2_image.get_rect(left=random.randint(DEFAULT_ZERO, DEFAULT_LEVEL2_SCREEN_WIDTH), top= DEFAULT_TOP_DOWN_NUMBER)
                    dy = DEFAULT_RANDOM_NUMBER
                    double_rocks.append((double_rock, dy))
                    score += DEFAULT_SCORE
                    DOUBLE_ATTACK_CHECK = True
                    DOUBLE_ATTACK_COUNT = DEFAULT_ZERO
                    explosion_sound.play()
                    break

              
            
    return score, DOUBLE_ATTACK_COUNT, DOUBLE_ATTACK_CHECK
    

# 종료 체크 함수
def handlingBranchMissGameOver(missed, gameOver):
#     if missed >= DEFAULT_END_MISS:
    if missed >= DEFAULT_END_MISS:
        gameOver = True
        pygame.mixer.music.stop()
        game_over_sound.play()

    if gameOver == True:
        GAME_END = handlingBranchGameOver()
        return GAME_END
        
# 공격 기본 함수 
def handlingBranchDefaultAttck(attacks):
    for attack in attacks:
        attack.top -= DEFAULT_ATTACK_SPPED        # 공격속도
        if attack.top < DEFAULT_ZERO:
            attacks.remove(attack)
    
# 비행기 이동 함수
def handlingBranchMoveFighter(fighter):
    if fighter.left < DEFAULT_ZERO:
        fighter.left = DEFAULT_ZERO
    elif fighter.right > SCREEN_WIDTH:
        fighter.right = SCREEN_WIDTH
        
# 기본 장애물 접촉 gameOver 함수 
def handlingBranchColliderectRocks(rocks, fighter, gameOver):
    for rock, dy in rocks:
        if rock.colliderect(fighter):
            gameOver = True
            pygame.mixer.music.stop()
            game_over_sound.play()

        if gameOver == True:
            GAME_END = handlingBranchGameOver()
            return GAME_END

# 기본 장애물 접촉 gameOver 함수 
def handlingBranchColliderectDoubleRocks(double_rocks, fighter, gameOver):
    for double_rock, dy in double_rocks:
        if double_rock.colliderect(fighter):
            gameOver = True
            pygame.mixer.music.stop()
            game_over_sound.play()    
        
        if gameOver == True:
            GAME_END = handlingBranchGameOver()
            return GAME_END
        

# 화면 이미지 전송 
def screenSendImage(rocks, double_rocks, attacks, fighter):    
    # 벽돌
    for rock, dy in rocks:
        screen.blit(wall_level_1_image, rock)
        
    # 더블 벽돌  
    for double_rock, dy in double_rocks:
        screen.blit(wall_level_2_image, double_rock)
        
    # 미사일
    for attack in attacks:
        screen.blit(attack_image, attack)
        
    # 비행기 
    screen.blit(fighter_image, fighter)
    
# 점수 
def handlingBranchScore():
    score_image = score_font.render('점수 {}'.format(TOTAL_SCORE), True, screenColor_Yellow)
    screen.blit(score_image,(10, 10))
    

# 게임 종료 
def handlingBranchGameOver():
    game_over_image = large_font.render('Game Over', True, screenColor_Red)
    screen.blit(game_over_image, game_over_image.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY))
    
    score_image = small_font.render('점수 {}'.format(TOTAL_SCORE), True, screenColor_Yellow)
    screen.blit(score_image, score_image.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY - DEFAULT_TOP_DOWN_NUMBER))
#     screen.blit(score_image, (10, 10))

    missed_image = small_font.render('놓친 운석 {}'.format(missed),  True, screenColor_Yellow)
    screen.blit(missed_image, missed_image.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY - (DEFAULT_TOP_DOWN_NUMBER * DEFAULT_TWO)))
#     screen.blit(missed_image, missed_image.get_rect(right=SCREEN_WIDTH - 10, top=10))
    
    GAME_END = True
    return GAME_END


In [6]:
"""
====================
시작화면 추가
====================
"""
def startGame():
    startEnter = False
    while True:
        mainText =   small_font.render('===== Fly Attack =====', True, screenColor_Blue)
        startText = small_font.render('시작하기 --> Key A', True, screenColor_Blue)
        middleText = small_font.render('=====            =====', True, screenColor_Blue)
        
        screen.blit(mainText,mainText.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY))
        screen.blit(startText,startText.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY - DEFAULT_TOP_DOWN_NUMBER))
        screen.blit(middleText,middleText.get_rect(centerx=DEFAULT_CENTERX, centery= DEFAULT_CENTERY - (DEFAULT_TOP_DOWN_NUMBER * DEFAULT_TWO)))
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit(0)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_a:
                    startEnter = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_a:
                    startEnter = False
                    
        if startEnter:
            break
        
    

In [7]:
   
# pygame.key.set_repeat(1, 1)

"""
====================
게임 루프 로직 (실제 로직)
====================
"""
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) 
clock = pygame.time.Clock() 

# 게임 시작 
startGame()

while True: 
    screen.blit(background_image, (0, 0))

    # 변수 업데이트
    event = pygame.event.poll() #이벤트 처리
    # 게임 기본 핸들링 처리 함수
    attacks, PYGAME_END = handlingBranchDefaultGame(event, attacks, PYGAME_END)
    if PYGAME_END == 1:
        break
    
    # 비행기 이동 로직
    fighter = handlingBranchDefaultFighter(fighter, gameOver)
    
#     if event.type == pygame.QUIT:
#         break
#     elif event.type == pygame.KEYDOWN:
#         if event.key == pygame.K_SPACE:
#             attack = pygame.Rect(attack_image.get_rect(centerx=fighter.centerx, top=fighter.top))
#             attacks.append(attack)
#             attack_sound.play()
        
#     pressed = pygame.key.get_pressed()
#     if pressed[pygame.K_LEFT] and not gameOver:
#         fighter.left -= DEFAULT_FIGHTER_KEY
#     elif pressed[pygame.K_RIGHT] and not gameOver:
#         fighter.left += DEFAULT_FIGHTER_KEY
    
    if not gameOver:
        # 장애물 로직
        missed = handlingBranchMissRock(rocks, missed)
        
        # 더블 장애물 로직 
        missed = handlingBranchMissDoubleRock(double_rocks, missed)
        
        # 종료 체크 로직
        GAME_END = handlingBranchMissGameOver(missed, gameOver)
        
        # 공격 로직
        handlingBranchDefaultAttck(attacks)

        # 비행기 이동 로직 
        handlingBranchMoveFighter(fighter)

        # 장애물 격파 로직 
        one_attack_tmp_score = handlingBranchAttackRock(rocks, attacks, score)
        TOTAL_SCORE += one_attack_tmp_score

        # 더블 장애물 로직             
        double_attack_tmp_score, double_attack_tmp_count, double_attack_tmp_chk = handlingBranchAttackDoubleRock(double_rocks, attacks, score, DOUBLE_ATTACK_COUNT, DOUBLE_ATTACK_CHECK)
        TOTAL_SCORE += double_attack_tmp_score
        DOUBLE_ATTACK_COUNT += double_attack_tmp_count
        if double_attack_tmp_chk == True:
            DOUBLE_ATTACK_COUNT = DEFAULT_ZERO
            DOUBLE_ATTACK_CHECK = False
            double_attack_tmp_chk = False
        
        # 기본 장애물 접촉 게임 종료
        GAME_END = handlingBranchColliderectRocks(rocks, fighter, gameOver)
        # 더블 장애물 접촉 게임 종료
        GAME_END = handlingBranchColliderectDoubleRocks(double_rocks, fighter, gameOver)


    # 화면 이미지 전송
    screenSendImage(rocks, double_rocks, attacks, fighter)
    
    handlingBranchScore()
    
    # 게임 종료 
    gameOver = GAME_END
    
    # 게임 종료 화면 
    if GAME_END == True:
        gameOver = handlingBranchGameOver()        
        
    pygame.display.update() 
    # FPS 프레임
    clock.tick(30) 

    
pygame.quit() 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SHJ\AppData\Local\Temp/ipykernel_13352/63282566.py", line 12, in <module>
    startGame()
  File "C:\Users\SHJ\AppData\Local\Temp/ipykernel_13352/2286634980.py", line 17, in startGame
    pygame.display.flip()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _

TypeError: object of type 'NoneType' has no len()